In [24]:
import sys
sys.path.append('../scripts')  
from models import *
from df_handling import *
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import warnings
import statistics
from sklearn.decomposition import PCA
warnings.filterwarnings("ignore")


In [20]:
#importing the datasets we want to test (you can change the dataframes you want to load accordingly)
greek2 = pd.read_csv('../data/greek_par_emb.csv')
greek2 = unravel_df(greek2, ['text_embedding_claim', 'text_embedding_par'])
cyprus2 = pd.read_csv('../data/cyprus_par_emb.csv')
cyprus2 = unravel_df(cyprus2, ['text_embedding_claim','text_embedding_par'])
check4facts2 = pd.read_csv('../data/check4facts_par_emb.csv')
check4facts2 = unravel_df(check4facts2, ['text_embedding_claim','text_embedding_par'])
#aggregating the claim id for each dataset
check4facts2['claim_id'] += int(greek2['claim_id'].iloc[-1]) +1
cyprus2['claim_id'] += int(check4facts2['claim_id'].iloc[-1]) +1
# #creating source labels for each dataset (optionally, not necessary for the classification)
# temp_df = pd.read_csv('../data/greek_web_scraping.csv')
# temp_df['source'] = temp_df['url'].apply(lambda x: 0 if "ellinikahoaxes" in x else (1 if "factcheckgreek" in x else None))
# greek1 = pd.merge(greek1, temp_df[['id','source', 'claim']], on='claim', how='inner')
# cyprus1['source'] = 3
# check4facts1['source'] = 2

#finally we create the dataframe that the model will be trained on (you can also change this accordingly)
df_train = pd.concat([greek2,cyprus2,check4facts2], ignore_index=True)
df_train = df_train[df_train.label<=1]
df_train.reset_index(drop=True, inplace=True)


In [21]:
#create the list for the holding the training vectors (claim plus n*paragraphs)
combined_array = list()
#create a list holding the classification label
label = list()
#for each unique claim id 
for i in df_train.claim_id.unique():
    #create a temp df holding information about a specific claim_id
    df = df_train[df_train.claim_id==i]
    try:
        # display(df)
        #store the claim embedding
        claim=np.array(df.iloc[0,8])
        #store the label
        label.append(df.iloc[0,9])
    except IndexError:
        continue
    # for each paragraph related to the claim
    par = []
    for j in range(len(df)):
        #concatenate all the paragraphs together
        par = np.concatenate([par,df.iloc[j,10]])
    #finally concatenate the related paragraphs with the claim
    claim = np.concatenate([claim, par])
    #store the final vector on an array
    combined_array.append(claim)
    

#we rearrange the combined array into an array format
arr = [combined_array[i] for i in range(len(combined_array))]

#make the train test split based on the previous arrays
X_train, X_test, y_train, y_test= train_test_split(arr  ,np.array(label).astype('int'), test_size=0.2, random_state=42)

#PCA feature reduction (optional)

# X_train_scaled, X_test_scaled = list(),list()

# for x_train in X_train:
#     x_train = np.array(x_train, dtype=np.float64)
#     x_train = x_train.reshape(1536, 3) # the format is transformed into a 1536shaped vector. The second number in a tuple is equal to 
#     pca = PCA(n_components = 1)        # the number of extra paragraphs plus one. In this example we have 2 extra pars plus one
#     X_train_scaled.append(pca.fit_transform(x_train))
    
# for x_test in X_test:
#     x_test = np.array(x_test, dtype=np.float64)
#     x_test = x_test.reshape(1536, 2)
#     pca = PCA(n_components = 1) 
#     X_test_scaled.append(pca.fit_transform(x_test))

#default classification
MLP(X_train, X_test, y_train, y_test)


#classification using the pca technique
#MLP(np.array(X_train_scaled).squeeze(), np.array(X_test_scaled).squeeze(), y_train, y_test)

,claim_id,title,body,most_similar_paragraph,harvest_date,url,most_similar_par_cos,claim,text_embedding_claim,label,text_embedding_par
0,0.0,"Στην εντατική ο υπουργός Άμυνας των ΗΠΑ, Lloyd...",Το Παρασκήνιο της Αγοράς Αφιέρωμα : 26 πολιτικ...,"ο υπουργός Άμυνας των ΗΠΑ, Lloyd Austin",2024-02-23,https://www.bankingnews.gr/diethni/articles/72...,0.6614,"Ο Υπουργός Άμυνας των ΗΠΑ, Lloyd Austin, φέρετ...","[-0.005461568478494883, -0.007872236892580986,...",0,"[-0.04415225610136986, -0.00244485423900187, -..."
